# Getting to know Typescript

Typescript compiles to another language: Javascript.

## Item 1: Understand the relationship between TS and JS

TS is a superset of JS. It uses `.ts` or `.tsx` extension file rather than `.js` or `.jsx`.
Every JS script is a TS script.

In [3]:
function greet(who: string) {
    console.log('Hello', who);
}

The code above won't work in a Javascript interpreter, as the `who: string` is a TS annotation.

In [4]:
let city = 'new york city';
console.log(city.toUppercase());

2:18 - Property 'toUppercase' does not exist on type 'string'. Did you mean 'toUpperCase'?


The code above is spotted by the TS type checker. In JS, it would raise a `TypeError`.
String has been detected using type inference.

In [ ]:
const states = [
    {name: 'Alabama', capital: 'Montgomery'},
    {name: 'Alaska', capital: 'Juneau'},
    {name: 'Arizona', capital: 'Phoenix'},
    // ...
];
for (const state of states) {
    console.log(state.capitol);
}

In JS, it would have returned 3 times `undefined`

What is make the mistake in the `states` variable?

In [ ]:
const states = [
    {name: 'Alabama', capitol: 'Montgomery'},
    {name: 'Alaska', capitol: 'Juneau'},
    {name: 'Arizona', capitol: 'Phoenix'},
    // ...
];
for (const state of states) {
    console.log(state.capital);
}

Now the warning is the opposite of what we want. To be clear, let's define an `inteface`.

In [ ]:
interface State {
    name: string;
    capital: string;
}
const states: State[] = [
    {name: 'Alabama', capitol: 'Montgomery'},
    {name: 'Alaska', capitol: 'Juneau'},
    {name: 'Arizona', capitol: 'Phoenix'},
    // ...
];
for (const state of states) {
    console.log(state.capital);
}

We now see clearly the error.

TS's type system models the runtime behavior of Javascript.

In [ ]:
const x = 2 + '3'; // OK, type is string
const y = '2' + 3; // OK, type is string

In [5]:
const a = null + 7; // Evaluates to 7 in JS
const b = [] + 12; // Evaluates to '12' in JS

1:11 - Operator '+' cannot be applied to types 'null' and '7'.
2:11 - Operator '+' cannot be applied to types 'undefined[]' and 'number'.


In [6]:
greet("tom", "baptiste")

1:14 - Expected 1 arguments, but got 2.


Sometimes, it goes over the JS's behavior

Even with type checking, errors can be raised at runtime.

In [7]:
const names = ['Alice', 'Bob'];
console.log(names[2].toUpperCase());

evalmachine.<anonymous>:5
exports.tsLastExpr = console.log(names[2].toUpperCase());
                                          ^

TypeError: Cannot read properties of undefined (reading 'toUpperCase')
    at evalmachine.<anonymous>:5:43
    at evalmachine.<anonymous>:7:3
    at sigintHandlersWrap (node:vm:268:12)
    at Script.runInThisContext (node:vm:127:14)
    at Object.runInThisContext (node:vm:305:38)
    at Object.execute (/home/tomperr/.nvm/versions/node/v16.13.1/lib/node_modules/tslab/dist/executor.js:162:38)
    at JupyterHandlerImpl.handleExecuteImpl (/home/tomperr/.nvm/versions/node/v16.13.1/lib/node_modules/tslab/dist/jupyter.js:219:38)
    at /home/tomperr/.nvm/versions/node/v16.13.1/lib/node_modules/tslab/dist/jupyter.js:177:57
    at async JupyterHandlerImpl.handleExecute (/home/tomperr/.nvm/versions/node/v16.13.1/lib/node_modules/tslab/dist/jupyter.js:177:21)
    at async ZmqServer.handleExecute (/home/tomperr/.nvm/versions/node/v16.13.1/lib/node_modules/tslab/dist/jupy

### Things to remember

TS is a superset of JS. It has a type system that models JS behaviour, but that goes beyound sometimes. 

## Item 2: Know which TS options you're using

Let's take the example of the code bellow:

In [11]:
function add(a, b) {
    return a + b;
}
add(10, null);

10


Should it transpile or not. It depends on the TS compiler options, that can be passed via command line or via config file `tsconfig.json`

```json
{
    "compilerOptions": {
        "noImplicitAny": true
    }
}
```

When we over the name of the function with the mouse, with `noImplicitAny: False`, it shows the `any` types

For new projects, we should start with `noImplicitAny` on, so that we write the types as we write the code.

`strictNullChecks` controls whether `null` and undefined are permissible values in every type

In [1]:
const x: number = null;

With `strictNullChecks` on, the code above would crash, and we would have been forced to use:

In [2]:
const x: number | null = null;

In [3]:
function toUpper(a: string | null) {
    return a?.toUpperCase() // with the ?, we assume it's not null
}

There are many other options.

### Things to remember

TS compiler includes several settings, via `tsconfig.json` or via command line. Aim to enable `strict` to get the most thorough checking that TS can offer.

## Item 3: Understand that code generation is independent of types

`tsc` compiler does two things: convert TS/JS to an older JS that works in browser; it checks code for type errors. These are unrelated: the types won't affect the JS that is generated.

### Compiling and type checking

It's wrong to say that "TS doesn't compile" because of errors. If it's valid JS, TS compiler will produce an output. It's more correct to say that it has errors or that "it doesn't type check"

In [5]:
const username = "tom";
name = 12
// valid JS, so a JS output will still be produced

2:1 - Cannot find name 'name'.


### You cannot check typescript types at runtime

In [6]:
interface Square {
    width: number;
}
interface Rectangle extends Square {
    height: number;
}
type Shape = Square | Rectangle;
function calculateArea(shape: Shape) {
    if (shape instanceof Rectangle) {
        // ~~~~~~~~~ 'Rectangle' only refers to a type,
        // but is being used as a value here
        return shape.width * shape.height;
        // ~~~~~~ Property 'height' does not exist
        // on type 'Shape'
    } else {
        return shape.width * shape.width;
    }
}

9:26 - 'Rectangle' only refers to a type, but is being used as a value here.
12:36 - Property 'height' does not exist on type 'Shape'.
12:36 - Property 'height' does not exist on type 'Square'.


Here, we use `Rectangle` TS type as a value, which is impossible because `Rectangle` exists only for compiler, not at runtime.

In [7]:
class Square {
    constructor(public width: number) {}
};

class Rectangle extends Square {
    constructor(public width: number, public height: number) {
        super(width)
    }
}

type Shape = Square | Rectangle;

function calculateArea(shape: Shape) {
    if (shape instanceof Rectangle) {
        return shape.height * shape.width;
    } else {
        return shape.width * shape.width;
    }
}

This code works because class `Rectangle` introduces both a `type` and a `value`

### Type operations cannot affect runtime values

In [8]:
function asNumber(val: number | string): number {
    return val as number;
}

generates this JS code

In [9]:
function asNumber(val) {
    return val;
}

To normalize values, we need to check their runtime type.

In [10]:
function asNumber(val: number | string): number {
    return typeof(val) === 'string' ? Number(val) : val;
}

### Runtime types may not be the same as declared types

In [12]:
function setLightSwitch(value: boolean) {
    switch (value) {
        case true:
            //turnLightOn();
            break;
        case false:
            //turnLightOff();
            break;
        default:
            console.log(`I'm afraid I can't do that.`);
    }
}

At runtime, `value` param may not be a boolean.

### You cannot overload a function based on typescript types

The only way to *provide* overloading is to do

In [17]:
function toAdd(a: number, b: number): number;
function toAdd(a: string, b: string): string;

function toAdd(a, b) {
    return a+b;
}

const three = toAdd(1,2) // number
const ten = toAdd("5", "5") // string
const zero = toAdd(0, "0") // error

10:14 - No overload matches this call.
10:14 - Overload 1 of 2, '(a: number, b: number): number', gave the following error.
10:14 - Argument of type 'string' is not assignable to parameter of type 'number'.
10:14 - Overload 2 of 2, '(a: string, b: string): string', gave the following error.
10:14 - Argument of type 'number' is not assignable to parameter of type 'string'.


### Typescript types have no effect on runtime performance

No impact because they are erased after transpilation. It's truly zero cost. Instead, there is build time. Also, the generated code may have some overhead.

### Things to remember

Code gen is independent of the type system. TS cannot affect the runtime behavior. It is possible to have a generated JS file even with errors. TS types are not available at runtime.

## Item 4: Get confortable with structural typing

"If it walks like a duck and talks lick a duck..."

In [18]:
interface Vector2D {
    x: number;
    y: number;
}

In [19]:
function calculateLength(v: Vector2D) {
    return Math.sqrt(v.x * v.x + v.y * v.y);
}

In [20]:
interface NamedVector {
    name: string;
    x: number;
    y: number;
}

In [22]:
const v: NamedVector = {name: "batman", x:1, y:2};
calculateLength(v);

2.23606797749979


No error is produced because `NamedVector` also has `x` and `y` properties, and TS recognizes it, even that we didn't describe the relationship between `Vector2D` and `NamedVector`

However, sometimes, it can produce bugs 

In [23]:
interface Vector3D {
    x: number;
    y: number;
    z: number;
}

In [24]:
function normalize(v: Vector3D) {
    const length = calculateLength(v)
    return {
        x: v.x / length,
        y: v.y / length,
        z: v.z / length,
    }
}

Here, `length` is computed with only `x` and `y` values, which is mathematically wrong.

It's easy to imagine that objects will be called with having the properties we have declared and no others (what we call "sealed" object). However, it TS / JS, objects are always "open"

In [26]:
function calculateLengthL1(v: Vector3D) {
    let length = 0;
    for (const axis of Object.keys(v)) {
        const coord = v[axis];
        // ~~~~~~~ Element implicitly has an 'any' type because ...
        // 'string' can't be used to index type 'Vector3D'
        length += Math.abs(coord);
    }
    return length;
}

This code can lead to bug, as the objects is not sealed and keys other than `x`, `y` and `z` can exist. Other properties may not be numbers.

In [28]:
const vec3d = {x: 1, y: 2, z: 3, name: "joker"};
calculateLengthL1(vec3d)

NaN


Some other suprises can happen with classes

In [29]:
class C {
    foo: string;
    constructor(foo: string) {
        this.foo = foo + "!"
    }
}

In [32]:
const c = new C("hey");
const d: C = {foo: "hey"}

In [36]:
console.log(c, d)

C { foo: 'hey!' } { foo: 'hey' }


Here, the logic in the constructor has been skipped, but as the properties / constructor signature matche, there is no error.

Structural typing is beneficial when writing test, it allows to mock easily.

In [38]:
interface Author {
    first: string;
    last: string;
}

interface DB {
    runQuery: (sql: string) => any[];
}

function getAuthors(database: DB): Author[] {
    const authorRows = database.runQuery("SELECT first, last FROM authors");
    return authorRows.map(row => ({first: row[0], last: row[1]}));
}

To mock the db during test, simply create an object with a custom `runQuery` method

In [42]:
const mockedDB: DB = {
    runQuery(sql: string): any[] {
        return [['Toni', 'Morrison'], ['Maya', 'Angelou']];
    }
}

### Things to remember

JS is duck typed and TS uses structural typing. Types are not sealed but open. We may not have only the properties we expect (even for classes). Use structural typing to facilitate unit testing.

## Item 5: Limit use of the any type

Typing is optional, it can be disabled at any time use `any` type

In [43]:
let age: number;
age = '12' as any

12


Even if it is really convenient, it also eliminates many of the advantages of using TS.

### There is no type safety with `any` types 

`age` is supposed to be a `number`, but is in fact a `string` because we used `any`. Let's unleash the chaos.

In [44]:
age += 1

121


### `any` lets you break contracts (signatures)

In [47]:
function calculateAge(birthDate: Date): number {
    return 1; // mock, just for example
}

let birthDate: any = '2022-11-29';
calculateAge(birthDate)

1


No error because of the use of `any` type

### There are no languages services for `any` types

IDE / code editor won't give you any hint when using `any` type (e.g. autocompletion)

In [49]:
let person: any = {first: "tom", last: "perr"};
// try to type `person.`, no available autocompletion 

It removes a lot of IDE features (not only autocomplete)

### Things to remember

`any` silences type checker, and also eliminates TS advantages. Avoid using it if possible.